# Описание

Классификация изображений на основе векторизации c использованием CLIP

## Зависимости

### Инициализация

In [2]:
import os
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split

In [3]:
! pip install kneed
! pip install kneed[plot]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
! pip install ftfy regex tqdm

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-th49ttzt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-th49ttzt
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
Defaulting to user installation because normal site-packages is not writeable


In [6]:
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

Torch version: 1.13.1+cu117


In [8]:
import clip

In [9]:
import cv2 as cv2
from google.colab.patches import cv2_imshow
import gdown
from natsort import natsorted, ns
from pathlib import Path
from random import sample, randint
from PIL import Image, ImageOps
from collections import Counter
from multiprocessing import Pool
from tqdm import tqdm
tqdm.pandas()

ModuleNotFoundError: No module named 'google.colab'

## Параметры

In [ ]:
# определяем колонку по которой будем связывать все таблицы
global_key = 'regNumber'

# путь к папке к изображениями из ГК
img_data_url = './data/img_for_clasifire/V__270523/'

In [ ]:
# ссылки 
url_dict = {
    'clear_data_list':'./out/gk_out/clear_data_list.csv',    
    'plot_list':'./out/gk_out/plot_list.csv',  
    'sample_objects':'./out/gk_out/sample_objects.csv',
    }

### Вспомогательные функции

In [ ]:
# для многоканальной обработки списка любой функцией
def to_pool(item_list, method):
    with Pool(processes=os.cpu_count()) as pool:
         rezult_list = []
         for res in tqdm(pool.imap(method, (i for i in item_list)),
                     total = len(item_list)):
             rezult_list.append(res)
    return rezult_list

In [ ]:
def to_tqdm(item_list, method):
  rezult_list = []
  for res in tqdm(map(method, (i for i in item_list)),
                     total = len(item_list)):
             rezult_list.append(res)
  return rezult_list

In [ ]:
# вывод изображений
def show_img(image_list, name = True, col = 5, cv2_img = False, suptitle='', title_key = global_key):
    row = len(image_list) // col + 1

    plt.figure(figsize=(6.4, 2.4*row))
    for i in range(0,len(image_list)):
        plt.subplot(row, col, i + 1)
        plt.suptitle(suptitle)
        plt.axis('off')
        if name:
           title = str(image_list[i][title_key])
        else:
           title =  'N' + str(i+1)
        plt.title(f"{title}")
        if cv2_img:
           plt.imshow(cv2.cvtColor(image_list[i]['img'], cv2.COLOR_BGR2RGB))
        else:
           plt.imshow(image_list[i]['img'])

In [ ]:
# считать список файлов из всех поддерикторий
# возвращает список имен файлов и уникальные расширения файлов
def get_file_list(path):
    rezult = []
    variant = set()
    for root, dirs, files in os.walk(path, topdown = False):
        for name in files:
            rezult.append(os.path.join(root, name))
            # если нет расширений у файлов добавляем None, иначе - название расширения
            file_extension = name.split('.')[1] if len(name.split('.')) > 1 else None
            variant.add(file_extension)
    return rezult, variant

In [ ]:
# на вход передается путь к имени файла и список id объектов. id соответствует первой части имени файла
# по маске id_*.jpg
def filter_fille_name(fille_path, id_list):
    fille_name = fille_path.split('/')[-1]
    return fille_name.split('.')[0].split('_')[0] in id_list

In [ ]:
# считать список файлов из всех поддерикторий
# возвращает список имен файлов и уникальные расширения файлов
def get_filter_file_list(path, name_list):
    rezult = []
    variant = set()
    for root, dirs, files in os.walk(path, topdown = False):
        for name in files:
            if not(filter_fille_name(name, name_list)):
               continue
            rezult.append(os.path.join(root, name))
            # если нет расширений у файлов добавляем None, иначе - название расширения
            file_extension = name.split('.')[1] if len(name.split('.')) > 1 else None
            variant.add(file_extension)
    return rezult, variant

In [ ]:
# %load ./src/functions/save2csv.py

# сохраняем в файл результаты
def save2csv(df, param='data'):
    file_name = './out/' + param + '_' + str(randint(100, 2000)) +'.csv'
    df.to_csv(file_name, index= False )


### Данные


Для дальнейшей работы используются результаты выгрузки данных ГК.
Файлы изображений должны именоваться по следующему шаблону:
regNumber_postfics.jpg, где

regNumber - регестрационный номер в ГК, это id служит для связи между изображением и предметом
postfics - число 1, 2 или 3, где 1 - обозначает основное изображение в карточке объекта в ГК, 2, 3 - остальные

In [ ]:
# возвращает id объекта из пути к файлу картинки и преффикс 1 или 2

# fille_path - путь к файлу
def setImgId(fille_path):
  fille_name = fille_path.split('/')[-1]
  return fille_name.split('.')[0].split('_')[0], fille_name.split('.')[0].split('_')[1]


In [ ]:
# возвращает id объекта из пути к файлу картинки

# fille_path - путь к файлу
def getImgId(fille_path):
  return int(setImgId(fille_path)[0])

In [ ]:
# возвращает преффикс 1 или 2 объекта из пути к файлу картинки

# fille_path - путь к файлу
def filterImgPreff(fille_path, pref_list = [1, '1']):
    fille_pref = setImgId(fille_path)[1]
    return fille_pref in pref_list

In [ ]:
# возвращает метку кластера для переданного изображения

# line - словарь вида {'img': изображение формата PIA/CV2, 'title':путь к файлу, 'regNumber':id по которому матчатся таблицы}
# loc_key = '' - название столбца по которому соединяются таблицы (если не указан - 'regNumber')
def setTrueLabel(line, loc_key = ''):
  key = loc_key if loc_key else global_key
  img_id = int(line[key])
  return valid_table_df.loc[valid_table_df[key] == img_id]['label_true'].tolist()[0]


In [ ]:
# конвертация в формат cv2
# возвращает словарь вида {'img':вектор изображения в формате CV2,'title':название}

# filename - полный путь к файлу
def file2cv2(filename):
     return {'img':cv2.imread(filename), 'title':filename}


In [ ]:
# конвертация в формат cv2
# возвращает словарь вида {'img':вектор изображения в формате PIL,'title':название}

# filename - полный путь к файлу
def file2PIL(filename):
     return {'img': Image.open(filename), 'title':filename}

In [ ]:
# скачать с гугл-диска таблицу в формате csv
# возвращает таблицу

# url - ссылка доступа, полученная с гугл-диска
def downloadFromDriverCSV(url, sep = ','):
   file_id=url.split('/')[-2]
   dwn_url='https://drive.google.com/uc?id=' + file_id
   df = pd.read_csv(dwn_url, encoding = 'utf8', sep = sep)

   return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Изображения и список ралевантных изображений

In [ ]:
# создаем список id ролевантных изображений
table_df = csv2DF('clear_data_list')
clear_id_list = table_df[global_key].tolist()

In [ ]:
# загрузка изображений 
# img_list - список изображений
# type_set - список расширений загруженных файлов для проверки результатов
img_list, type_set = get_file_list(img_data_url)
type_set

Два варианта - либо создаем список картинок без класса 0 отфильтровав img_list если работаем с неочищенным списком. Либо сразу грузим только нужные картинки (пока закоментировала этот вариант)

In [ ]:
#img_list_clear, type_set_clear = get_filter_file_list(google_foolder, clear_id_list)
clear_id_list = table_df[global_key].tolist()

In [ ]:
# переводим картинки из списка в формат PIL
image_list = to_pool(img_list, eval('file2PIL'))

In [ ]:
# оставляем тольк первые изображения
image_list = list(filter(lambda item: filterImgPreff(item['title'], [1, '1']), image_list))
print(len(image_list))

In [ ]:
data_img =  pd.DataFrame(image_list)

In [ ]:
# добавляем колонку с id по которому матчим таблицы
image_num_list = to_pool(data_img['title'].tolist(), eval('getImgId'))
data_img[global_key] = image_num_list

#### Эталонные картинки для центройдов

In [ ]:
# соединяет таблицу изображений и таблицу сюжетов по ключевому полю

# df_plot - датафрейм с сюжетами, df_img - датафрейм с изображениями
# loc_key = '' - название столбца по которому соединяются таблицы (если не указан - 'regNumber')
def setPlotImg(df_plot, df_img, loc_key = ''):
  key = loc_key if loc_key else global_key
  key_plot_list = np.unique(df_plot[key].tolist())
  df_temp = df_img.loc[df_img[key].isin(key_plot_list)]

  return df_plot.merge(df_temp, on=key)


In [ ]:
# создаем список сюжето
plot_table_df = csv2DF('plot_list')
plot_table_df =  plot_table_df[[global_key, 'name']]

In [ ]:
# создаем словарь сюжетов
plot_list = np.unique(plot_table_df['name'].tolist())

plot_dictionary = {plot_list[i]:str(i) for i in range(0, len(plot_list))}
# добавляем номер сюжета по словарю
plot_table_df['plot_id'] = plot_table_df['name'].map(plot_dictionary)

In [ ]:
plot_table_df.head()

In [ ]:
# У нас совпадают ID картинок для двух сюжетов
# В каталоге это лист с несколькими гравюрами https://goskatalog.ru/portal/#/collections?id=34450495
plot_table_df[plot_table_df[global_key] == 34251283]

In [ ]:
# Выкидываю из списка пока оба сюжета
plot_table_df = plot_table_df.loc[~plot_table_df[global_key].isin([34251283])]

In [ ]:
print(f'Всего сюжетов {len(plot_table_df)}')

#### Валидационная выборка

In [ ]:
# валидационная выборка
valid_table_df = csv2DF('sample_objects')

In [ ]:
# убираем из валидационной выборки connectedness = 0 и plot = NaN
values_list = [0.0]
valid_table_df = valid_table_df[~valid_table_df['connectedness'].isin(values_list)]
valid_table_df = valid_table_df[valid_table_df['plot'].notna()]
valid_table_df = valid_table_df[['name' ,'regNumber','plot']]

# добавляем номер сюжета по словарю
valid_table_df['label_true'] = valid_table_df['plot'].map(plot_dictionary)



In [ ]:
valid_table_df.head()

In [ ]:
# удаляем афишу выставки Русские гравюры 18 века по рисункам Михаила Махаева, поскольку сюжета такого нет )
valid_table_df = valid_table_df[valid_table_df['label_true'].notna()]

In [ ]:
valid_id_list = valid_table_df[global_key].tolist()
plots_in_valid_table_df = np.unique(valid_table_df['label_true'].tolist())

In [ ]:
# Распределение сюжетов, представленных в выборке
plots_in_valid_table_dict = dict(Counter(valid_table_df['plot']))

# сюжеты, для которых в выборке больше 2 объектов
common_plots_in_valid_table = dict(filter(lambda x: x[1] >= 2, plots_in_valid_table_dict.items()))

In [ ]:
print(f'Всего сюжетов: {len(plot_table_df)}')
print(f'Количество сюжетов, представленных в выборке: {len(plots_in_valid_table_df)}')
print(f'Количество сюжетов, представленных в выборке для которых количество объектов больше или = 2:{len(common_plots_in_valid_table)}')

In [ ]:
lemit = 10
print(f'{lemit} самых распространенных сюжета:')
print(*list(dict(sorted(plots_in_valid_table_dict.items(), key=lambda item: item[1], reverse=True)).items())[0:lemit], sep = '\n')

Выводы:
- в валидационной выборке представленны не все сюжеты
- почти половина сюжетов представленны в выборке 1 объектом, следовательно такие алгоритмы кластаризации, как DBSCAN отнесут их в "мусор"

## Обработка картинок

### Тензор

In [ ]:
# импорт из clip нужного
import requests
from transformers import CLIPProcessor, CLIPModel
from transformers import AutoProcessor, CLIPModel

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# получает тензор изображения с помощью clip-model
# возвращает обект типа tensor()

# image - вектор изображения формата PIL
def image_features_get(image):
    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)

    return image_features.detach()

In [ ]:
features_list = to_pool(data_img['img'].tolist(), eval('image_features_get'))

In [ ]:
#reshape
features_list_reshape = [t.detach().numpy().reshape(-1) for t in features_list]


In [ ]:
data_img['features'] = features_list_reshape
data_img.sample(5)

In [ ]:
# создаем список ключевых изображений
data_key_img = setPlotImg(plot_table_df, data_img)

In [ ]:
# проверяем, что нашли все ключевые изображения
key_plot_list_in_img = data_key_img[global_key].tolist()
plot_table_df[~plot_table_df[global_key].isin(key_plot_list_in_img)]

In [ ]:
# отделяем трейн и тест выборки
data_img_valid = data_img.loc[data_img[global_key].isin(valid_id_list)].copy()

data_img_test = data_img.loc[~data_img[global_key].isin(valid_id_list)].copy()
# отделяем "чистую выборку" (без conntents=0)
data_img_clear = data_img_test.loc[data_img_test[global_key].isin(clear_id_list)].copy()
print(f'Длина валидационной выборки: {len(data_img_valid)}. Длина общей выборки: {len(data_img_test)}. Длина очищенной выборки: {len(data_img_clear)}')

In [ ]:
# для трейна прописываем label
label_true = to_pool(data_img_valid.to_dict(orient='records'), eval('setTrueLabel'))
data_img_valid['label_true'] = label_true
data_img_valid.head()

In [ ]:
train = data_img_valid['features'].tolist()
probs =  data_img_test['features'].tolist()
probs_clear = data_img_clear['features'].tolist()

In [ ]:
labels_true = data_img_valid['label_true'].tolist()

## Общее для кластаризации

In [ ]:
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import normalize, StandardScaler

In [ ]:
global_score_funcs = [
    #("V-measure", metrics.v_measure_score),
    #("Rand index", metrics.rand_score),
    ("ARI", metrics.adjusted_rand_score),
    #("MI", metrics.mutual_info_score),
    #("NMI", metrics.normalized_mutual_info_score),
    ("AMI", metrics.adjusted_mutual_info_score),
    ("Однородность, Полнота, V-мера", metrics.homogeneity_completeness_v_measure)
]

In [ ]:
# выводим метрику в табличной форме
def metricsRezult_forDF(metric_list):
    rezult_dict = {}

    for item in metric_list:
        if isinstance(item['score'], tuple):
              score_title_list = item['score_name'].split(',')
              for i in range(len(item['score'])):
                  rezult_dict[score_title_list[i]] = item['score'][i]
        else:
            rezult_dict[item['score_name']]=item['score']
    return rezult_dict


In [ ]:
# возвращает результаты проверки кластеризации различными метриками
# x- предсказанные метки, y - истинные метки
# score_funcs_list - список кортежей формата (название_функции, функция)
def getMetricsRezult(x, y, score_funcs_list):
    rezult_list = []
    for func in score_funcs_list:
        score_func = func[1]
        score_name = func[0]
        rezult = score_func(y, x)
        rezult_list.append({'score_name': score_name, 'score':rezult})
    return rezult_list

In [ ]:
# выводит на печать результаты кластаризации

# labels - список полученных лейблов
def print_сlustering_param(labels):
  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  n_noise_ = list(labels).count(-1)
  print(f'Количество кластеров : {n_clusters_}. Количество шума: {n_noise_} ')

In [ ]:
# выводит на печать словарь вида {кластер:количество элементов в нем}

# labels - список лейблов
def print_claster(df, label, sampel_size = 30, suptitle = '', cv2_img = False, name = False):
  rezult_df = df[df['labels'] == label].to_dict(orient='records')
  if len(rezult_df) > sampel_size:
     show_img(sample(rezult_df, sampel_size), name = name, suptitle = suptitle, cv2_img = cv2_img)
  else:
     show_img(rezult_df, name = name, suptitle = suptitle, cv2_img = cv2_img)

In [ ]:
# выводит на печать изображения по всем кластерам
# df - датафрейм с столбцами 'img' (вектор изображения PIL или CV2), 'title' (заголовок), 'labels' (присвоенный класс)
# sampel_size = 30 - максимальный размер выборки изображений,
# cv2_img = False - булеан использовать формат cv2 или нет,
# name = False -  булеан печатать имя картинки или нет,
def print_img_in_classters(df, cv2_img = False, sampel_size = 30, suptitle = '', name = False):
    for k in np.unique(df['labels']):
        df_temp = df[df['labels'] == k]
        suptitle = '-'.join([str(k), str(len(df_temp))])
        print_claster(df, k, suptitle = suptitle, sampel_size = sampel_size, cv2_img = cv2_img , name = name)

In [ ]:
# выводит на печать словарь вида {кластер:количество элементов в нем}

# labels - список лейблов
def print_claster_count(labels):
    return  dict(Counter(labels))

In [ ]:
# выводит на печать картинки из переданного списка, согласно кластаризации
# или печатает только один класс, если есть ключ кластера

# img_df - дата фрейм с колонками 'img' - типа PIC/CV, 'title' - имя картинки = ссылка на картинку в выгрузке
# labels - список лейблов
# key - ключ кластера
# printOneClaster = False, - печатаем все или только выбранный кластер
# sampel_size = 30, suptitle = '', cv2_img = False, name = False - параметры печати
def printVizual(img_df, labels, key=False, printOneClaster = False, sampel_size = 30, suptitle = '', cv2_img = False, name = False):
    df_temp = pd.DataFrame({'img': img_df['img'], 'title': img_df['title'], 'labels':labels})
    if printOneClaster:
       print_claster(df_temp, key, sampel_size = sampel_size, suptitle = suptitle, cv2_img = cv2_img, name =name)
    else:
       print_img_in_classters(df_temp, sampel_size = sampel_size, suptitle = suptitle, cv2_img = cv2_img, name =name)


In [ ]:
def save_claster_rezult(img_df, labels, param):
    df_rezult = pd.DataFrame({global_key: img_df[global_key], 'title': img_df['title'], 'img': img_df['img'], 'labels':labels})
    save2csv(df_rezult, param=param)

## DBSCAN

### Функции

In [ ]:
from sklearn.cluster import DBSCAN


In [ ]:
# применяет DBSCAN с переданными параметрами
# возвращает список полученных лейблов

# X - список фичей,
# eps=0.1, min_samples=4, metric='cosine' - параметры DBSCAN,
# (подробней https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN)
# scale  -   булево значение стандартизировать или нет перед применение DBSCAN
# norm  -   булево значение нормализовать или нет перед применение DBSCAN
def get_dbscan_label_test(X, eps=0.1, min_samples=4, metric='cosine', scale = True, norm = True):
    db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
    if scale:
       scaler = StandardScaler()
       X = scaler.fit_transform(X)
    if norm:
       X = normalize(X)
    db.fit(X)
    return db.labels_

##### Табличный вывод

In [ ]:
# вывод DF с результатами работа DBSCAN в табличной форме
def rezult_DF_DBSCAN(klaster_list, printTitle = True, printCounter = True, printScore = False):
    rezult_list = []

    for item in klaster_list:
        row_dict = {}

        if printTitle:
           row_dict.update({'title':item['title']})

        row_dict.update({
            'data':item['data_type'],
            'min_samples':item['min_samples'],
            'eps':item['eps']
        })

        count = dict(Counter(item['labels']))
        n_noise_ = list(item['labels']).count(-1)
        row_dict.update({'class':len(count) - 1,  'noise':n_noise_})
        if printScore:
           current_metric_list = item['score']
           row_dict.update(metricsRezult_forDF(current_metric_list))
        if printCounter:
           row_dict.update({'counter':count})

        rezult_list.append(row_dict)

    return pd.DataFrame(rezult_list)

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
from kneed import KneeLocator

In [ ]:
# метод для нахождения оптимального eps для DBSCAN,
# используется для дальнейшего построения графика "колена"
# возвращает средние дистанции между точками набора, полученные методом NearestNeighbors

# X - список фичей,
# num = 21 -  параметр earestNeighbors, количество соседей = MinPts, которое будет использоваться в DBSCAN,
# print_plt = False - булево значение печать график или нет
# scale  -   булево значение стандартизировать или нет перед применение NearestNeighbors
# norm  -   булево значение нормализовать или нет перед применение NearestNeighbors

def get_neighbors(X, num = 11, print_plt = False, scale = True, norm = True):
  neigh = NearestNeighbors(n_neighbors=num)
  if scale:
       scaler = StandardScaler()
       X = scaler.fit_transform(X)
  if norm:
       X = normalize(X)
  nbrs = neigh.fit(X)
  distances, indices = nbrs.kneighbors(X)
  distances = np.sort(distances[:,num-1], axis=0)
  if print_plt:
     fig = plt.figure(figsize = (18, 9))
     plt.plot(distances)
     plt.xlabel('Points')
     plt.ylabel('Distance')

  return distances

In [ ]:
# возвращает значенеи "колена"

# dist - дистанции между соседями, полученные методом NearestNeighbors
# print_plt = False - булево значение печать график или нет
def get_kneeLocatore(dist, S=1.0, online=False, print_plt = False):
    x, y = np.arange(len(dist)), dist
    kneedle = KneeLocator(x, y, S=S, curve="convex", direction="increasing", online=online)
    if print_plt:
       kneedle.plot_knee_normalized()

    knee_point = kneedle.knee
    elbow_point = kneedle.elbow
    return knee_point, elbow_point

In [ ]:
#  результаты кластерезации методом DBSCAN с различными параметрами
# возвращает лист словарей вида {'eps':значение eps, 'min_samples':минимальный размер кластера, 'n_clusters_':количество кластеров, 'n_noise_':количество шума}
# отсортированный по возрастанию шума

# X - список фичей,
# param_list - список словарей вида {'eps':значение eps, 'min_samples':минимальный размер кластера}
# min_claster_count = 2 - минимальное количество кластеров, которое должно получится в выборке
# getScore = False - печатать оценку кластеризации
# Y='' - список классов для валидации
# score_funcs_list = '' - список кортежей формата (название_функции, функция) (по умолчанию global_score_funcs)
def get_dbscan_test(X, param_list, min_claster_count = 2, getScore = False, Y='', score_funcs_list = ''):
    rezult = []
    for row in param_list:
       min_samples = row['min_samples']
       eps = row['eps']
       labels =  get_dbscan_label_test(X, eps=eps, min_samples=min_samples)
       n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
       n_noise_ = list(labels).count(-1)
       if getScore:
          score_funcs_list = score_funcs_list if score_funcs_list else global_score_funcs
          rezult.append({'eps':eps, 'min_samples':min_samples, 'n_clusters_':n_clusters_, 'n_noise_':n_noise_, 'score_list':getMetricsRezult(labels, Y, score_funcs_list)})
       else:
          rezult.append({'eps':eps, 'min_samples':min_samples, 'n_clusters_':n_clusters_, 'n_noise_':n_noise_})

    rezult = list(filter(lambda item: item['n_clusters_'] > min_claster_count, rezult))
    rezult = sorted(rezult, key=lambda item: item['n_noise_'])

    return rezult


In [ ]:
# печатает результаты кластерезации методом DBSCAN с различными параметрами

# X - список фичей,
# param_list - список словарей вида {'eps':значение eps, 'min_samples':минимальный размер кластера}
# getScore = False - печатать оценку кластеризации
# Y='' - список классов для валидации
# score_funcs_list = '' - список кортежей формата (название_функции, функция) (по умолчанию global_score_funcs)
def print_dbscan_test(X, param_list, getScore = False, Y='', score_funcs_list = ''):
    for row in param_list:
       min_samples = row['min_samples']
       eps = row['eps']
       labels =  get_dbscan_label_test(X, eps=eps, min_samples=min_samples)

       n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
       n_noise_ = list(labels).count(-1)
       print(f'eps={eps}, min_samples={min_samples}')
       print(f'Количество кластеров : {n_clusters_}. Количество шума: {n_noise_} ')
       print(print_claster_count(labels))
       if getScore:
          score_funcs_list = score_funcs_list if score_funcs_list else global_score_funcs
          score_list = getMetricsRezult(labels, Y, score_funcs_list)
          print(*score_list, sep='\n')
       print('+'*100)

In [ ]:
# создает список параметров и проводит для них тестирование DBSCAN
# в зависимости от флага выводит на печать результаты
# возвращает лист словарей вида {'eps':значение eps, 'min_samples':минимальный размер кластера, 'n_clusters_':количество кластеров, 'n_noise_':количество шума}
# отсортированный по возрастанию шума

# X - список фичей,
# eps_list - список eps для которых будет формироваться лист параметров
# samples_list = [3, 4] - список min_samples для которых будет формироваться лист параметров
# print=False - булево значение печатать результаты тестов
# getScore = False - печатать оценку кластеризации
# Y='' - список классов для валидации
# score_funcs_list = '' - список кортежей формата (название_функции, функция) (по умолчанию global_score_funcs)
def set_dbscan_param(X, eps_list, samples_list = [3, 4], print=False, getScore = False, Y=''):
    min_samples_list = []
    for i in samples_list:
        min_samples_list = min_samples_list + len(eps_list) * [i]
    param_list = pd.DataFrame(list(zip(eps_list*len(samples_list), min_samples_list)),columns =['eps', 'min_samples']).to_dict(orient='records')
    if print:
       print_dbscan_test(X, param_list, getScore = getScore, Y=Y)
    else:
       return get_dbscan_test(X, param_list, getScore = getScore, Y=Y)

### На валидационной выборке

#### Оптимальные параметры

In [ ]:
best_eps = 0.541
best_sample = 2

labels_1 = get_dbscan_label_test(train, eps = best_eps, min_samples=best_sample)
print_сlustering_param(labels_1)
print(print_claster_count(labels_1))
getMetricsRezult(labels_1, labels_true, global_score_funcs)

In [ ]:
test_rezult_list_DBSCAN = [
    {
                'title':'с лучшими значениями метрик на валидационной',
                'eps': best_eps,
                'min_samples': best_sample,
                'data_type': 'валидационная',
                'labels': labels_1,
                'score': getMetricsRezult(labels_1, labels_true, global_score_funcs)
               }
]

In [ ]:
df_table = rezult_DF_DBSCAN(test_rezult_list_DBSCAN, printScore = True, printCounter =False, printTitle =False)
df_table

#### Визуализация

Видно, что распределение по кластерам более равномерное.

In [ ]:
printVizual(data_img_valid, labels_1, sampel_size = 200)

In [ ]:
#bad
plan_num = 0
printVizualCollag(data_img_valid, labels_1, plan_num, size = (256, 256), cols = 6)

In [ ]:
plan_num = 9
printVizual(data_img_valid, labels_1, printOneClaster = True, key = plan_num, sampel_size = 200, suptitle = 'План Петербурга 1753.')

In [ ]:
ramka_klaster = 0
printVizual(data_img_valid, labels_1, printOneClaster = True, sampel_size = 200, key = ramka_klaster, suptitle = 'Гравюры с рамкой')

In [ ]:
list_klaster = 2
printVizual(data_img_valid, labels_1, printOneClaster = True, key = list_klaster, sampel_size = 200, suptitle = 'Гравюра на листе')

Для тестовой выборки вариант с eps=0.541 и min_sampel = 2 показывает лучшие значения по метрикам.

- Хорошо определил кластер с 'План Петербурга 1753' (почти все 20 из 21 в выборке)
- Проблема - воспринимает как отдельные кластеры гравюры с рамкой и подписью и небольшие гравюры посередине листа в независимости от сюжета

#### Сохранение результатов с лучшими параметрами класстаризации

In [ ]:
save_claster_rezult(data_img_valid, labels_1, param='DBSCAN_valid')

### На тесте

###### Оптимальные параметры

Поскольку мы знаем количество сюжетов в выборке (95) оринтируясь на количество кластеров оптимальным по соотношению количество кластеров/количество шума выглядит

Для не очищенной выборки:
eps = 0.382, min_samples = 2
(Количество кластеров: 92, Количество шума: 259)

Для очищенной:
eps=0.387, min_samples=2
(Количество кластеров : 94. Количество шума: 241)


In [ ]:
best_eps_test =  0.382
best_sample_test = 2
best_eps_test_clear =  0.387
best_sample_test_clear = 2

In [ ]:
test_rezult_DBSCAN = {'Выборка с лучшими значениями метрик на валидационной выборки':get_dbscan_label_test(probs, eps = best_eps, min_samples=best_sample),
               'Выборка чистая с лучшими значениями метрик на валидационной выборки':get_dbscan_label_test(probs_clear, eps = best_eps, min_samples=best_sample),
               'Выборка с оптимальными значениями eps/min_samples на общей выборке':get_dbscan_label_test(probs, eps = best_eps_test, min_samples=best_sample_test),
               'Выборка чистая с оптимальными значениями eps/min_samples на чистой выборке':get_dbscan_label_test(probs_clear, eps = best_eps_test_clear, min_samples=best_sample_test_clear)}

In [ ]:
list_test_rezult_DBSCAN = []
for k, v in test_rezult_DBSCAN.items():
    print(f'{k}')
    count = Counter(v)
    print(f'Количество классов {len(count) - 1}') # поскольку один класс это "мусор"
    print(count)
    list_test_rezult_DBSCAN.append({'title':k, 'labels':v, 'count':dict(count)})

In [ ]:
test_rezult_list_DBSCAN = [{
                'title':'с лучшими значениями метрик на валидационной',
                'eps': best_eps,
                'min_samples': best_sample,
                'data_type': 'полная',
                'labels': list_test_rezult_DBSCAN[0]['labels']
               },
               {
                'title':'с лучшими значениями метрик на валидационной',
                'eps': best_eps,
                'min_samples': best_sample,
                'data_type': 'чистая',
                'labels':list_test_rezult_DBSCAN[1]['labels']
               },
               {
                'title':'с лучшими значениями метрик на тесте',
                'eps': best_eps_test,
                'min_samples': best_sample_test,
                'data_type': 'полная',
                'labels':list_test_rezult_DBSCAN[2]['labels'],
               },
               {
                'title':'с лучшими значениями метрик на тесте',
                'eps': best_eps_test_clear,
                'min_samples': best_sample_test_clear,
                'data_type': 'чистая',
                'labels':list_test_rezult_DBSCAN[3]['labels'],
               }
              ]

In [ ]:
df_table = rezult_DF_DBSCAN(test_rezult_list_DBSCAN, printCounter =False)
df_table

Очевидно, что значения eps/min_samples, полученные на валидационной выборке, плохо себя показали на общей.

###### Визуализация

In [ ]:
# визуализируем результаты - Выборка с оптимальными значениями eps/min_samples на общей выборке
printVizual(data_img_test, list_test_rezult_DBSCAN[2]['labels'], suptitle = list_test_rezult_DBSCAN[2]['title'], sampel_size = 200)

In [ ]:
# визуализируем результаты - Выборка чистая с оптимальными значениями eps/min_samples на чистой выборке
printVizual(data_img_clear, list_test_rezult_DBSCAN[3]['labels'], suptitle = list_test_rezult_DBSCAN[0]['title'], sampel_size = 200)

Также как на валидационной выборке, лучше всего выделяются планы и карты. Есть несколько кластеров объединенных по форме (гравюра на странице, открытка, печатный текст), а не по сюжету.

###### Сохранение результатов с лучшими параметрами класстаризации

In [ ]:
print(
      list_test_rezult_DBSCAN[2]['title'],
      list_test_rezult_DBSCAN[3]['title'], sep = '\n')

In [ ]:
save_claster_rezult(data_img_test, list_test_rezult_DBSCAN[2]['labels'], param='DBSCAN_full')
save_claster_rezult(data_img_clear, list_test_rezult_DBSCAN[3]['labels'], param='DBSCAN_clear')

## KMeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# применяет KMeans с переданными параметрами
# возвращает список полученных лейблов

# X - список фичей,
# k - количество кластеров,
# n_init= 'auto', random_state=1 - параметры KMeans
# scale  -   булево значение стандартизировать или нет перед применение KMeans
# norm  -   булево значение нормализовать или нет перед применение KMeans
# use_centroids = False - использовать переданный массив центройдов
# centroids = '' - центройды
def get_KMeans_labels(X, k=0, scale = True, norm = True, use_centroids = False, centroids = '', init="k-means++", n_init= 'auto', random_state=1):
    if use_centroids:
       # если передаем центройды
       n_clusters= k if k != 0 else len(centroids)
       kmeans = KMeans(n_clusters=max(2, n_clusters), init=centroids, n_init=1, random_state=random_state)
    else:
       kmeans = KMeans(init= init, n_clusters=max(k,2), n_init=n_init, random_state=random_state)
    if scale:
       scaler = StandardScaler()
       X = scaler.fit_transform(X)
    if norm:
       X = normalize(X)
    kmeans.fit(X)
    return kmeans.labels_

### Подбор параметров

In [ ]:
# нахождение оптимального количества кластеров для KMeans,
# методом силуэта
# возвращает средние значение силуэта для выбранного количества кластеров

# X - список фичей,
# num = 10 -  параметр KMeans, количество примененеий метода для рассчета среднего значения силуэта,
# scale  -   булево значение стандартизировать или нет перед применение KMeans
# norm  -   булево значение нормализовать или нет перед применение KMeans
def get_KMeans_silhouette_score(X, k, num = 10, scale = True, norm = True, init="k-means++", metric='cosine'):
    kmeans = KMeans(init= init, n_clusters=max(k,2), n_init= 'auto', random_state=num)
    if scale:
       scaler = StandardScaler()
       X = scaler.fit_transform(X)
    if norm:
       X = normalize(X)
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    silhouette_avg = silhouette_score(X, cluster_labels, metric=metric)

    return silhouette_avg


#### Табличный вывод

In [ ]:
def rezult_DF_test(klaster_list, num = 10, scale = True, norm = True, printCounter = True):
    rezult_list = []

    for item in klaster_list:
        row_dict = {}
        count = dict(Counter(item['labels']))
        X = item['data']
        row_dict.update({'title':item['title'],'class':len(count), 'data':item['data_type'],'counter':count})
        if printCounter:
           row_dict.update({'counter':count})

        row_dict['silhouette_score'] = get_KMeans_silhouette_score(X, len(count), num = num, scale = scale, norm = norm)

        rezult_list.append(row_dict)

    return pd.DataFrame(rezult_list)

In [ ]:
def rezult_DF(klaster_dict_list, X, y, num = 10, scale = True, norm = True, title_list = [], printCounter = True, printSilhouet = True):
    rezult_list = []
    for klaster_dict in klaster_dict_list:
      for k, v in klaster_dict.items():
          row_dict = {}

          row_dict.update({'class':k})

          if printCounter:
             row_dict.update({'counter':dict(Counter(v))})
          if printSilhouet:
             row_dict.update({'silhouette_score':get_KMeans_silhouette_score(X, k, num = num, scale = scale, norm = norm)})
          current_metric_list = getMetricsRezult(v, y, global_score_funcs)
          row_dict.update(metricsRezult_forDF(current_metric_list))

          rezult_list.append(row_dict)
    df_rezult = pd.DataFrame(rezult_list)
    if len(title_list) != 0:
       df_rezult.insert(0, 'title', title_list)


    return df_rezult

In [ ]:
# рассчитывает значение силуэта для различных k в заданном диапазоне
# возвращаем словарь вида {количество кластеров:значение силуэта}

# X - список фичей,
# start = 10, end = 110, step_list = [10, 5, 1] - диапазон в которых проверяме значения k и шаг с которым выполняем проверку
# set_len = 3 - размер возвращаемого словаря
# scale  -   булево значение стандартизировать или нет перед применение KMeans
# norm  -   булево значение нормализовать или нет перед применение KMeans
def get_klasters_set(X, start = 10, end = 110, step_list = [10, 5, 1], set_len = 3, scale = True, norm = True):
    current_start = start
    current_end = end
    for step in step_list:
        # считаем для каждого k значение силуэта
        k_dict = {max(k, 2): get_KMeans_silhouette_score(X, k, scale = scale, norm = norm) for k in range(current_start, current_end, step)}
        # сортируем результаты по значениям силуэта
        k_dict = dict(sorted(k_dict.items(), key=lambda item: item[1], reverse = True))

        # print(k_dict)
        # берем два k с самыми высокими значениями и сортируем их по возрастанию k
        # таким образом получаем новые значения current_start и current_end
        param = sorted(list(k_dict.items())[0:2])
        current_start, current_end = param[0][0], param[1][0] + 1

    # берем первые значения из списка (по умолчанию - 3)
    rezult = list(k_dict.items())[0:set_len]
    # получем список кластеров
    #return [i[0] for i in rezult]
    return rezult

### На валидационной выборке

In [ ]:
# не нашла более разумного способа, поэтому отдельно считаем возможное количество кластеров от 2 до 10 и от 10 до 100
k_dict = get_klasters_set(train) + get_klasters_set(train, start = 0, end = 10, step_list = [1])
k_list = [i[0] for i in k_dict]
k_list

In [ ]:
# Поскольку знаем реальное количество сюжетов (95), добавим в конец списка это значение
k_list.append(len(data_key_img))
# Поскольку знаем реальное количество сюжетов в валидационной выборке (70), добавим в конец списка это значение
k_list.append(len(plots_in_valid_table_df))

In [ ]:
labels_dict = dict(zip(k_list, [get_KMeans_labels(train, k) for k in k_list]))

In [ ]:
df_table = rezult_DF([labels_dict], X = train, y = labels_true, printCounter = False)
df_table

In [ ]:
for k, v in labels_dict.items():
    print(f'Количество классов {k}')
    print(Counter(v))
    print(getMetricsRezult(v, labels_true, global_score_funcs))

###### Оптимальные параметры

In [ ]:
best_score_k = k_list[0]

print(f'Количество классов {best_score_k}')
score_list = getMetricsRezult(labels_dict[best_score_k], labels_true, global_score_funcs)
print(*score_list, sep='\n')
print(f'Количество классов {len(plots_in_valid_table_df)}')
score_list = getMetricsRezult(labels_dict[len(plots_in_valid_table_df)], labels_true, global_score_funcs)
print(*score_list, sep='\n')

###### Визуализация

In [ ]:
printVizual(data_img_valid, labels_dict[best_score_k], suptitle = 'Лучший score')

In [ ]:
printVizual(data_img_valid, labels_dict[len(plots_in_valid_table_df)], suptitle = 'По известному количесву сюжетов')

##### Центройды

In [ ]:
centroids = pd.DataFrame(list(data_key_img['features'])).to_numpy()

Поскольку у нас отличается количество сюжетов в валидационной выборке и общее количество сюжетов, сделаем вариант только с центройдами из валидационной выборке.

In [ ]:
data_key_img_short = data_key_img.loc[data_key_img['plot_id'].isin(plots_in_valid_table_df)]
centroids_short = pd.DataFrame(list(data_key_img_short['features'])).to_numpy()
len(data_key_img_short)

In [ ]:
labels_with_centroids_short = get_KMeans_labels(X = train, k = len(centroids_short), use_centroids = True, centroids=centroids_short)

In [ ]:
# сравниваем на том же количестве классов, но без установленных центройдов
print(f'Количество классов {len(plots_in_valid_table_df)} с установленными центройдами')
print(Counter(labels_with_centroids_short))
score_list_centroids_short = getMetricsRezult(labels_with_centroids_short, labels_true, global_score_funcs)
print(*score_list_centroids_short, sep='\n')

print(f'Количество классов {len(plots_in_valid_table_df)}')
print(Counter(labels_dict[len(plots_in_valid_table_df)]))
score_list = getMetricsRezult(labels_dict[len(plots_in_valid_table_df)], labels_true, global_score_funcs)
print(*score_list, sep='\n')

print(f'Количество классов ({best_score_k}) с лучшими показателями метрики  без центройдов')
print(Counter(labels_dict[best_score_k]))
score_list = getMetricsRezult(labels_dict[best_score_k], labels_true, global_score_funcs)
print(*score_list, sep='\n')


In [ ]:
temp_dict_list = [
              {len(centroids_short): labels_with_centroids_short},
              {len(plots_in_valid_table_df): labels_dict[len(plots_in_valid_table_df)]},
              {best_score_k: labels_dict[best_score_k]}
]
temp_title_list = [
    'C центройдами',
    'То же количество кластеров, без центройдов',
    'С лучшими показателями метрики без центройдов'
]
df_table = rezult_DF(temp_dict_list, X = train, y = labels_true,  title_list = temp_title_list, printCounter = False)
df_table

#### Визуализируем

In [ ]:
printVizual(data_img_valid, labels_with_centroids_short, suptitle = 'С установленными центройдами')

In [ ]:
printVizual(data_img_valid, labels_dict[len(plots_in_valid_table_df)], suptitle = 'Без центройдов')

#### Сохранение результатов с лучшими параметрами класстаризации

In [ ]:
save_claster_rezult(data_img_valid, labels_dict[best_score_k], param='KMeans_valid')
save_claster_rezult(data_img_valid, labels_with_centroids_short, param='KMeans_valid_centr')

### На тесте

In [ ]:
# найдем лучшее значение силуэта для полной и на чистой выборке
k_dict_test = get_klasters_set(probs) + get_klasters_set(probs, start = 0, end = 10, step_list = [1])
k_list_test = [i[0] for i in k_dict_test]

k_dict_test_clear = get_klasters_set(probs_clear) + get_klasters_set(probs_clear, start = 0, end = 10, step_list = [1])
k_list_test_clear = [i[0] for i in k_dict_test_clear]

best_score_k_test = k_list_test[0]
best_score_k_test_clear = k_list_test_clear[0]

In [ ]:
test_rezult = {'Выборка с установленными центройдами': get_KMeans_labels(X = probs, k = len(centroids), use_centroids = True, centroids=centroids),
               'Выборка с количеством кластеров по числу сюжетов':get_KMeans_labels(X = probs, k = len(data_key_img)),
               'Выборка с количеством кластеров с лучшими значениями метрик на валидационной выборке':get_KMeans_labels(X = probs, k = best_score_k),
               'Выборка с лучшими значениями силуэта на тесте':get_KMeans_labels(X = probs, k = best_score_k_test),

               'Выборка чистая с установленными центройдами': get_KMeans_labels(X = probs_clear, k = len(centroids), use_centroids = True, centroids=centroids),
               'Выборка чистая с количеством кластеров по числу сюжетов':get_KMeans_labels(X = probs_clear, k = len(data_key_img)),
               'Выборка чистая с количеством кластеров с лучшими значениями метрик на валидационной выборке':get_KMeans_labels(X = probs_clear, k = best_score_k),
               'Выборка чистая с лучшими значениями силуэта на тесте': get_KMeans_labels(X = probs_clear, k = best_score_k_test_clear)
               }

In [ ]:
list_test_rezult = []
i = 0
for k, v in test_rezult.items():
    print(f'{i}. {k}')
    count = Counter(v)
    print(f'Количество классов {len(count)}')
    print(count)
    list_test_rezult.append({'title':k, 'labels':v, 'count':dict(count)})
    i+=1


In [ ]:
test_rezult = [{
                'title':'C центройдами',
                'data': probs,
                'data_type': 'полная',
                'labels': list_test_rezult[0]['labels']
               },
               {
                'title':'C количеством кластеров = числу сюжетов',
                'data': probs,
                'data_type': 'полная',
                'labels': list_test_rezult[1]['labels']
               },
               {
                'title':'C количеством кластеров = лучшими значениями метрик на валидационной выборке',
                'data': probs,
                'data_type': 'полная',
                'labels':  list_test_rezult[2]['labels']
               },
               {
                'title':'C лучшими значениями силуэта на тесте',
                'data': probs,
                'data_type': 'полная',
                'labels': list_test_rezult[3]['labels']
               },
               {
                'title':'Выборка чистая с центройдами',
                'data': probs_clear,
                'data_type': 'чистая',
                'labels': list_test_rezult[4]['labels']
               },
               {
                'title':'Выборка чистая количеством кластеров = сюжетов',
                'data': probs_clear,
                'data_type': 'чистая',
                'labels': list_test_rezult[5]['labels']
               },
               {
                'title':'Выборка чистая с количеством кластеров с лучшими значениями метрик на валидационной выборке',
                'data': probs_clear,
                'data_type': 'чистая',
                'labels': list_test_rezult[6]['labels']
               },
               {
                'title':'Выборка чистая с лучшими значениями силуэта на тесте',
                'data': probs_clear,
                'data_type': 'чистая',
                'labels': list_test_rezult[7]['labels']
               }
              ]

In [ ]:
df_table = rezult_DF_test(test_rezult)
df_table

#### Визуализируем


In [ ]:
# 'Выборка с установленными центройдами'
printVizual(data_img_test, list_test_rezult[0]['labels'], suptitle = list_test_rezult[0]['title'])

In [ ]:
# 4. Выборка чистая с установленными центройдами
printVizual(data_img_clear, list_test_rezult[4]['labels'], suptitle = list_test_rezult[4]['title'])

In [ ]:
# 6. Выборка чистая с количеством кластеров с лучшими значениями метрик на тесте
printVizual(data_img_clear, list_test_rezult[6]['labels'], suptitle = list_test_rezult[6]['title'])

In [ ]:
#7. Выборка чистая с лучшими значениями силуэта на тесте
printVizual(data_img_clear, list_test_rezult[7]['labels'], suptitle = list_test_rezult[7]['title'])

#### Сохранение результатов с лучшими параметрами класстаризации

In [ ]:
print(
      list_test_rezult[0]['title'],
      list_test_rezult[2]['title'],
      list_test_rezult[3]['title'],

      list_test_rezult[4]['title'],
      list_test_rezult[6]['title'],
      list_test_rezult[7]['title'],
      sep = '\n')

In [ ]:
save_claster_rezult(data_img_test, list_test_rezult[0]['labels'], param='KMeans_full_centr')
save_claster_rezult(data_img_test, list_test_rezult[2]['labels'], param='KMeans_full_best')
save_claster_rezult(data_img_test, list_test_rezult[3]['labels'], param='KMeans_full_siluet')

save_claster_rezult(data_img_clear, list_test_rezult[4]['labels'], param='KMeans_clear_centr')
save_claster_rezult(data_img_clear, list_test_rezult[6]['labels'], param='KMeans_clear_best')
save_claster_rezult(data_img_clear, list_test_rezult[7]['labels'], param='KMeans_clear_siluet')


In [ ]:
save_claster_rezult(data_img_test, list_test_rezult[0]['labels'], param='KMeans_full_centr')